# Case Study 7:  Newton's Method

Part of the purpose of this case study is to gain some experience with aspects of numerical precision.  The Sage docs for the various options for real and complex numbers are [here](https://doc.sagemath.org/html/en/reference/rings_numerical/index.html).  For now, just take a moment to internalize the fact that there's no way to keep a real number in a computer.

Newton's method is an iterative method to find a root to an equation.  For example, given $f(x) = x^2 - 5$, start with any "guess" such as $x_0=4$, and then iterate like so:
$$
x_i = x_{i-1} - \frac{f(x)}{f'(x)}.
$$
Write a small program/function that implements this for $f(x) = x^2 - 5$ and real numbers.  It should spit out the iterations $x_0, x_1, \ldots$ for as many as you want (the number of iterates to show should be an argument).

In [16]:
def newton( z, n ): 
    # z = initial guess
    # n = number of iterates to show


Try it!

In [ ]:
newton(4, 4)

Ok, if you didn't force your numbers to be real, you are actually getting rational numbers out, which becomes very complicated-looking very fast.  Instead, make this take place in the *real numbers*.  To make a rational or integer `z` into a real number in sage, you do `RR(z)`.  Call this updated version of your function `newton_real`.

In [18]:
def newton_real( z, n ): 
    # x = initial guess
    # n = number of iterates to show


In [ ]:
newton_real(4, 4)

Looks good, let's go further!

In [ ]:
out = newton_real(4, 100)
# let's just view the tail
out[-8:] # this looks at the last 8 entries, i.e. entry -8, -7, ... end=-1

Compare to the *actual* roots of the equation (I trust you can solve it!).  

Great!  (I hope.)

BTW, why does this work?  Calculus homework!  A good way to see it is to draw a picture of the iteration process on a graph.  I've done this for you just because I couldn't resist.

In [57]:
def newton_picture( z, n ): 
    # green = parabola
    G = plot( x^2 - 5, color="green", xmin = -5, xmax = 5 )
    z = RR(z)
    # blue = initial root guess and tangent to curve at that x-coordinate
    G += plot( + (z^2 - 5) + 2*z*(x - z), color="blue", xmin = -5, xmax = 5 )
    G += point( (z, 0), color="blue", xmin = -5, xmax = 5, size = 20)
    G += point( (z, z^2 - 5), color="blue", xmin = -5, xmax = 5, size = 20)
    for i in range(n):
        z = z - ( z^2 - 5 )/(2*z)
        # red = intersection of that blue line with x-axis as new root guess
        G += plot( + (z^2 - 5) + 2*z*(x - z), color="red", xmin = -5, xmax = 5)
        G += point( (z, 0), color="red", xmin = -5, xmax = 5, size = 20)
        G += point( (z, z^2 - 5), color="red", xmin = -5, xmax = 5, size = 20)
    return G

In [ ]:
newton_picture( 4, 1 ).show( xmin = -5, xmax = 5, ymin = -20 )

## Polynomials, symbolic variables

Ok, now let's modify our newton function.  The new one will take in any polynomial.  To do this, we need to be comfortable with polynomials and derivatives.  Check this out!  Try other polynomials.

In [ ]:
f = x^2 - 5
f.derivative()

Sage is a little weird in this regard.  You can deal with polynomials as symbolic objects, or elements of polynomial rings.  When you want a symbolic objects, you need to define variables.  The variable `x` comes preloaded in Sage.  Try this instead:

In [ ]:
f = y^2 - 5
f.derivative()

So `x` is special!  When you want to make more things symbolic variables, you have to declare them.

In [ ]:
x,y = var('x,y') 
f = y^2 - x^2 + x*y
f.derivative(x) # I have to say with respect to which variable

There's a quirk here in `x,y = var('x,y')`.  This declares the names as well as the variables (hence the repetition).
There's technically a difference between how you refer to variables and how Sage writes them to the screen, these can differ:
* the leftmost x,y are the variables (as you will type them to Sage)
* the rightmost x,y are how Sage will type them to you.

In the next cell, try changing only one of the two and see how confusing it is.  You can mostly forget this remark until it bites you.  Also, you might want to put this back to normal before you move on and get confused later.

In [ ]:
x,y = var('x,z') 
f = y^2 - x^2 + x*y
f.derivative(x)

Here's how you can evaluate polynomials.  Try evaluating a two-variable polynomial.

In [ ]:
f = x^2 - 5
f(x = 2)

Note:  if you declare a polynomial ring like `R.<x> = ZZ[x]`, then `x` is no longer a symbolic variable, but is instead an element of a polynomial ring.  Often this doesn't matter, until it does.  But notice the repetition again: the possible difference between how you write the variable and Sage does.

## Newton Iteration for any polynomial

Ok, enough of that.  Now make `newton_iteration` that takes in a single variable symbolic polynomial `f`.   (Let's call the new one `newton_iteration`; I'll try to avoid repeat names because it can lead to "footgunning"... I'll let you figure that one out).  

In [62]:
def newton_iteration( f, z, n ): 
    # f = poly to find root of
    # z = initial guess
    # n = number of iterates to show


In [ ]:
newton_iteration( x^2-5, 4, 4 )

In [ ]:
# check out this polynomial!
f = (x-1)*(x-1.00000000000001); 
f

In [ ]:
iters = newton_iteration( f, 10, 150 )
iters[-8:]

In [ ]:
# I'm just going to expand it out, no precision lost here, right?
g = f.expand()
g

In [ ]:
iters = newton_iteration( g, 10, 150 )
iters[-8:]

Wait, that's definitely weird.  What's going on?  It was the *same* polynomial!!

## Computers can't handle real numbers

What's happening is that in the first case, when we plug into `f`, we are plugging into the factored polynomial and then multiplying the factors.  In the second case, we are plugging into `g`, the already expanded polynomial.  It's a different sequence of events.  They are mathematically equivalent in the abstract world of polynomials over the real numbers, but not in Sage.  For the answer, evaluate the next cell.

In [ ]:
RR

That means that `RR` by default has 53 bits of precision and any extra digits are lost.  So there's rounding error.  And it's a very finicky thing to keep track of what types of operations lose precision at what rate, even when mathematically two options are equivalent!  A computer cannot store real numbers, because real numbers, in their platonic form, have infinite precision.

So what do we do?  We track precision.  For this reason, Sage has a whole bunch of different options for real (and complex) numbers.  Check out the following cells.

In [ ]:
RealField(10)

In [ ]:
RDF # double = 64 bits

In [ ]:
RIF

Interval arithmetic is a solution to the precision problem.  The idea is to track the possible error.  So for example, try this:

In [ ]:
RIF(sqrt(2))

Do you see the question mark?  What that is saying is that the true square root of $2$ lies between 1.41421356237309 and 1.41421356237310.  Actually -- achtung!! -- they are stored internally in hex, and the decimal printed to the screen is a rounding.  You can rely on them to be correct internally, but the actual endpoints are stored in base 16. 

So elements of RIF are actually intervals.  So you can actually specify them that way.  Let's see our square root in its internal glory:

In [ ]:
a = RIF(sqrt(2))
print(a.upper(), a.lower())

Now here's a very cool trick!  (Using pari under the hood.)  What is this doing??

In [ ]:
a.algdep(5) 

Ok, so fixed precision is dangerous, and interval arithmetic is frequently the solution.  You can use interval arithmetic via RIF as if the things you are dealing with are numbers, not intervals, but there's some weird gotchas, like comparisons `==`, `<=` etc. (I'll let you read Sage manuals if you want).  Copy `newton_iteration` here as `newton_iteration_RIF` and just cast `z` to RIF instead of RR.

In [149]:
def newton_iteration_RIF( f, z, n ): 
    # f = poly to find root of
    # z = initial guess
    # n = number of iterates to show


Now try the two polynomials we dealt with above.  You may need to cast the coefficients into RIF.  What happens?

In [ ]:
iters = newton_iteration_RIF( (x-RIF(1))*(x-RIF(1.00000000000001)), 4, 25)
iters[-8:]

In [ ]:
iters = newton_iteration_RIF( x^2 - RIF(2.00000000000001)*x + RIF(1.00000000000001), 4, 13)
iters[-8:]

It may not be satisfying but the point is that it didn't *lie* to you.

## But which root?

Your challenge now is to write a newton iteration function that works over the complex numbers.  [These](https://doc.sagemath.org/html/en/reference/rings_numerical/index.html) are your tools.  By default Sage understands both `i` and `I` as the complex upper half plane square root of -1 (unless you redefined one of them!).

In [38]:
def newton_iteration_Whatevertheheckthisoneiscalledyouchoose( f, z, n ): 
    # f = poly to find root of
    # z = initial guess
    # n = number of iterates to show


In [ ]:
newton_iteration_Whatevertheheckthisoneiscalledyouchoose( (x-1)*(x-2)*(x-3), i, 30 )

Now, write a function which will determine, for an input complex value z, which of the three roots of `(x-1)(x-2)(x-3)` the Newton iteration process will eventually approach.  (To be completely mathematically correct about this probably requires some more mathematical theory of Newton iteration, so do something reasonable.)

In [39]:
def which_root( z ): 


In [ ]:
which_root( 1+3*i ) # should be 3

And now, create a plot of the complex plane where each position is coloured by which root of the polynomial you approach with Newton iteration.  Let's stick to three roots.  So colour the location x+iy green (or whatever) if running Newton iteration with initial value x+iy approaches root 1, blue if it approaches root 2, etc.  Sage is probably *not* the right tool for this type of sophistocated plotting, so I'd do the simplest thing and fill a matrix with the results and then `plot(matrix, cmap='viridis')`.  But if you want to use an excellent tool for this, you could look into [shadertoy](https://www.shadertoy.com/).

In [ ]:
plot(m, cmap='viridis')

Now that you've done that, reward yourself by checking out [Wikipedia's gallery](https://en.wikipedia.org/wiki/Newton_fractal).